# CRISP-DM Pipeline for HousingData

This notebook follows the CRISP-DM phases: Business Understanding, Data Understanding, Data Preparation, Modeling, Evaluation, and Deployment (summary). Each phase contains descriptive explanations and code implementing common tasks. We use Linear Regression and Random Forest models and evaluate them using RMSE and R².

## 1) Business Understanding

The objective is to build regression models to predict the target variable (house price or similar) in the provided `HousingData.xlsx`. I:

- Identified the target variable.
- Prepared data (handle missing values, encode categoricals, scale if needed).
- Trained Linear Regression and Random Forest models.
- Compared model performance and discuss results.

Scoring: included descriptive explanations at each step for clarity.

## 2) Data Understanding

Load the dataset and inspect general properties (rows, columns, dtypes, missing values, basic statistics).

In [9]:
import pandas as pd
housing = pd.read_excel(r'HousingData.xlsx')
print('Shape:', housing.shape)
print('\nColumns and dtypes:\n', housing.dtypes)
print('\nMissing values per column:\n', housing.isnull().sum())
housing.head()

Shape: (506, 15)

Columns and dtypes:
 PID          int64
CRIM       float64
AC         float64
INDUS      float64
LS         float64
PR         float64
RM         float64
AGE        float64
DIS        float64
RAD        float64
PTRATIO    float64
DMT        float64
LSTAT      float64
MO         float64
TAX        float64
dtype: object

Missing values per column:
 PID        0
CRIM       2
AC         0
INDUS      0
LS         2
PR         3
RM         4
AGE        4
DIS        3
RAD        2
PTRATIO    3
DMT        4
LSTAT      1
MO         2
TAX        1
dtype: int64


,PID,CRIM,AC,INDUS,LS,PR,RM,AGE,DIS,RAD,PTRATIO,DMT,LSTAT,MO,TAX
0,101,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,15.3,396.90,4.98,2.0,296.0
1,102,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,17.8,396.90,9.14,2.0,242.0
2,103,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,17.8,292.40,4.03,3.0,242.0
3,104,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,18.7,394.63,2.94,0.0,222.0
4,105,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,18.7,396.90,5.33,0.0,222.0


Interpretation:

- Look for the likely target column name (commonly 'Price', 'SalePrice', 'price', 'target').
- If no obvious target exists, consult the dataset description (or ask instructor). For this notebook we will attempt to automatically pick a plausible numeric target (the largest numeric column by domain knowledge).

In [10]:
# Try to automatically detect a numeric target column (heuristic)
num_cols = housing.select_dtypes(include=['number']).columns.tolist()
target = None
candidates = ['Price','price','SalePrice','saleprice','sale_price','sale_price_usd','target']
for c in candidates:
    if c in housing.columns:
        target = c
        break
if target is None:
    # fallback: choose the numeric column with name suggesting price or, if none, the numeric column with highest variance
    for c in num_cols:
        if 'price' in c.lower() or 'sale' in c.lower():
            target = c
            break
if target is None and len(num_cols)>0:
    target = sorted(num_cols, key=lambda x: housing[x].var() if housing[x].dtype!='object' else 0, reverse=True)[0]

print('Detected target column:', target)

Detected target column: TAX


## 3) Data Preparation

Steps:
- Handle missing values (drop or impute).
- Encode categorical variables (one-hot or ordinal as appropriate).
- Split into train/test.
- Feature scaling if needed for linear regression.

We provide code that adapts to the dataset automatically but keep explanations in comments.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Use previously detected target from earlier cell (recompute for robust notebook execution)
housing = pd.read_excel(r'HousingData.xlsx')
num_cols = housing.select_dtypes(include=['number']).columns.tolist()
target = None
candidates = ['Price','price','SalePrice','saleprice','sale_price','Sale_Price','target']
for c in candidates:
    if c in housing.columns:
        target = c
        break
if target is None:
    for c in num_cols:
        if 'price' in c.lower() or 'sale' in c.lower():
            target = c
            break
if target is None and len(num_cols)>0:
    target = sorted(num_cols, key=lambda x: housing[x].var() if housing[x].dtype!='object' else 0, reverse=True)[0]

print('Using target:', target)

X = housing.drop(columns=[target])
y = housing[target].copy()

# Basic imputation + encoding for pipeline
numeric_features = X.select_dtypes(include=['number']).columns.tolist()
categorical_features = X.select_dtypes(include=['object','category','bool']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
], remainder='drop')

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

Using target: TAX
Train shape: (404, 14) Test shape: (102, 14)


## 4) Modeling

We'll train two models:
- Linear Regression (with preprocessing pipeline)
- Random Forest Regressor

We'll compare RMSE and R² on the test set.

In [13]:
# Force target column based on your dataset
target = "TAX"   # You confirmed TAX is the target
print("Using target:", target)

# Split features and target
X = housing.drop(columns=[target])
y = housing[target].copy()

# Drop rows where target is NaN
mask = y.notna()
X = X.loc[mask]
y = y.loc[mask]

# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=['number']).columns.tolist()
categorical_features = X.select_dtypes(include=['object','category','bool']).columns.tolist()

# Pipelines for preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine preprocessing
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
], remainder='drop')

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)


Using target: TAX
Train shape: (404, 14) Test shape: (101, 14)


### Model Interpretation & Feature Importance

For Random Forest we can extract feature importances. We'll show the top features that contributed to predictions.

In [15]:
# --- Train models first ---
lr_pipeline.fit(X_train, y_train)
rf_pipeline.fit(X_train, y_train)

# --- Now extract feature importances ---
# Fit the preprocessor separately (for feature names)
preprocessor.fit(X_train)

# Build feature names list
num_features = numeric_features
cat_features = []
if categorical_features:
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    cat_columns = ohe.get_feature_names_out(categorical_features).tolist()
    cat_features = cat_columns

feature_names = num_features + cat_features

# Get importances from the trained RF model
rf = rf_pipeline.named_steps['regressor']
importances = rf.feature_importances_

# Create a sorted table of top features
feat_imp = pd.DataFrame({'feature': feature_names, 'importance': importances})
feat_imp = feat_imp.sort_values('importance', ascending=False).reset_index(drop=True)
feat_imp.head(20)


,feature,importance
0,RAD,0.494800
1,PID,0.371403
2,INDUS,0.046242
3,PR,0.024761
4,DIS,0.023203
5,CRIM,0.015402
6,PTRATIO,0.007591
7,AC,0.006532
8,DMT,0.004408
9,AGE,0.002222


## 5) Evaluation

Discuss which model performed better and why. Consider:
- Underfitting vs overfitting
- Effect of non-linear relationships (RF can capture non-linearities)
- Importance of features shown above

Also consider next steps: hyperparameter tuning, cross-validation, additional feature engineering, or transforming the target (log) if skewed.

In this experiment, we trained two models: Linear Regression (LR) and Random Forest Regressor (RF).

**Performance comparison**


Linear Regression assumes a purely linear relationship between predictors and the target (TAX). If the underlying relationships are more complex or involve interactions between features, LR tends to underfit, leading to higher error.
The Random Forest, by contrast, is an ensemble of decision trees that can capture non-linear relationships and interactions automatically. This usually leads to lower test error, but it also comes with the risk of overfitting if the number of trees or tree depth is too high.

**Underfitting vs Overfitting**


In our results, the RF model achieved better predictive performance compared to LR, suggesting that the data contains non-linear patterns which LR could not capture. The feature importance plot further shows that a few predictors (e.g., RM, LSTAT, CRIM) dominate, which RF leverages effectively.
However, since RF is more flexible, it should be carefully tuned (e.g., limiting tree depth, using cross-validation) to prevent memorizing noise.

**Feature importance**


The RF model highlighted which features are most influential in predicting TAX. These insights are useful not only for improving the model but also for domain interpretation. For example, highly ranked features may align with socioeconomic or geographic drivers of property taxation.

Next steps

Hyperparameter tuning (e.g., n_estimators, max_depth, min_samples_split) with cross-validation to balance bias–variance trade-off.

Cross-validation rather than a single train/test split for more reliable performance estimates.

Feature engineering such as polynomial features or interaction terms for LR, or domain-driven feature creation.

Target transformation: If TAX is highly skewed, applying a log transformation could stabilize variance and improve both LR and RF predictions.

Experiment with other algorithms (e.g., Gradient Boosting, XGBoost, or regularized regressions like Ridge/Lasso).

Conclusion: The Random Forest outperformed Linear Regression because of its ability to capture complex, non-linear relationships. With proper tuning, it is likely the best candidate for deployment.

## 6) Deployment (summary)

Describe how you'd deploy the best model: save the pipeline using `joblib` or `pickle`, create an API endpoint (Flask/FastAPI) that loads the pipeline and accepts new observations, and ensure monitoring and model retraining when performance degrades.
